In [2]:
import pandas as pd
import re

In [3]:
df = pd.read_csv('/Users/daishiwei/Desktop/課程/文字探勘/data/20221218_餐廳彙整.csv')

In [95]:
#檢查是否在兩年內
#配合pd.df.map
#輸入：文字（評論時間）
#輸出：是否在兩年內 是：1 否：0
def time_ana(x):
    if re.search('a\syear\sago',x):
        intwoyear = 1
    elif re.search('\syears\sago',x):
        intwoyear = 0
    else:
        intwoyear = 1
    return intwoyear
#分開中文跟英文
#輸入：文字（評論內容
#輸出： 英文部分的評論, 中文部分的評論
def eng_split(x):
    t_s = re.sub('[\n]','',x)
    t_s_ch = re.sub('[a-zA-Z\s\d]','',t_s)
    t_s_ch = re.sub('[\W]','',t_s_ch)
    try:
        idx = re.search(t_s_ch[0],t_s).span()
        t_s_eng = t_s[:idx[0]]
    except:
        return t_s
    t_s_eng = re.sub('Translated by Google','',t_s_eng)
    t_s_eng = re.sub('(Original)','',t_s_eng)
    t_s_eng = re.sub('\(\)','',t_s_eng)
    return t_s_eng
def ch_split(x):
    t_s = re.sub('[\n]','',x)
    t_s_ch = re.sub('[a-zA-Z\s\d]','',t_s)
    t_s_ch = re.sub('[\W]','',t_s_ch)
    return t_s_ch

def del_dul(data, del_col= ['餐廳名稱','評論人', '評論內容']):
#根據 del_col裡包含的欄位的重複內容，進行清除
    df_del_re = data.drop_duplicates(subset = del_col)
    return df_del_re

In [5]:
#清除重複資料，標準是在同一間店、同一個人、同樣的留言
df = del_dul(df)

#標記是否在兩年內
df['是否在兩年內'] = df['評論時間'].map(time_ana)
#根據是否在兩年內進行篩選
df_time_pro = pd.DataFrame(df[df['是否在兩年內'] == 1])


In [ ]:
#清除評論為空的對象
df_clean =df_time_pro.dropna(axis=0,how='all',subset=['評論內容'])

In [98]:
#將評論內容分為英文、中文兩部分
df_clean['英文留言']= df_clean['評論內容'].map(eng_split)
df_clean['中文留言']= df_clean['評論內容'].map(ch_split)

斷詞

In [ ]:
#載入ckip
from ckiptagger import data_utils, construct_dictionary, WS, POS

In [ ]:
#載入模型
ws = WS("./data")
pos = POS("./data")

In [ ]:
#將傳入的列表中的每個評論斷詞、標注詞性，傳回斷詞列表、詞性列表
def ckip_tag_fun(review):
    sentence_list = review
    word_sentence_list = ws(
    sentence_list,
    )
    pos_sentence_list = pos(word_sentence_list)
    return word_sentence_list, pos_sentence_list

In [ ]:
ws_result, pos_result = ckip_tag_fun(df_clean['中文留言'])
df_clean['斷詞結果'],df_clean['詞性標注'] = ws_result,pos_result

In [ ]:
#釋放記憶體
del ws
del pos

情意分析

In [1]:
#載入nltk\vader
import nltk
#nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/daishiwei/nltk_data...


In [97]:
#將傳入的評論進行情意分析
def vader_fun(x):
    sid = SentimentIntensityAnalyzer()
    score = sid.polarity_scores(x)
    return score

In [107]:
#對每個英文評論的情意分析分數進行分類，大於0.05分為正面、小於-0.0.5的為負面、在此之間是中性
def sen_encode(x):
    if x['compound'] >0.05:
        return '正面'
    elif 0.05 >x['compound'] > -0.05:
        return '中性'
    elif -0.05 > x['compound']:
        return '負面'

In [100]:
#對每個英文評論進行情意分析
df_clean['情感分析'] = df_clean['英文留言'].map(vader_fun)

In [108]:
#對每個英文評論的情意分析分數進行分類
df_clean['情感分析結果分類'] = df_clean['情感分析'].map(sen_encode)

In [115]:
#印出總體情感分析分數的分類結果的總和
x = df_clean['情感分析結果分類'].value_counts()
print(x)
print(f'正面比例：{x.iloc[0]/(x.sum())}')
print(f'負面比例：{x.iloc[1]/(x.sum())}')
print(f'中性比例：{x.iloc[2]/(x.sum())}')

正面    122279
中性     36806
負面     16952
Name: 情感分析結果分類, dtype: int64
正面比例：0.6946210171725262
負面比例：0.20908104546203354
中性比例：0.09629793736544022


In [116]:
#根據每家店印出情感分析分數的分類結果的總和
df_clean.groupby('餐廳名稱')['情感分析結果分類'].value_counts()

餐廳名稱           情感分析結果分類
1010湘 (南港中信店)  正面          83
               負面          17
               中性          15
112巷牛排         正面          89
               中性          32
                           ..
龍翔園餐廳          中性          31
               負面          10
龟覓の鍋物          正面          80
               中性           4
               負面           3
Name: 情感分析結果分類, Length: 2860, dtype: int64

星等分析

In [121]:
#計算總體星等
print('所有資料：')
print('總體星等（未清除空值）：{}'.format(df['評論人給星'].sum()/df.shape[0]))
df_del_na = df.dropna(axis=0,how='all',subset=['評論內容'])
print('總體星等（清除空值）：{}'.format(df_del_na['評論人給星'].sum()/df_del_na.shape[0]))
#兩年內、未清除
print('兩年內資料')
print('總體星等（未清除空值）：{}'.format(df_time_pro['評論人給星'].sum()/df_time_pro.shape[0]))
print('總體星等（清除空值）：{}'.format(df_clean['評論人給星'].sum()/df_clean.shape[0]))


所有資料：
總體星等（未清除空值）：4.199818494442069
總體星等（清除空值）：4.152662786332281
兩年內資料
總體星等（未清除空值）：4.274607708832778
總體星等（清除空值）：4.199769374527814


In [173]:
#計算各家店星等
print('所有資料：')
print('總體星等（未清除空值）：{}'.format(df.groupby('餐廳名稱')['評論人給星'].sum()/df.groupby('餐廳名稱')['餐廳名稱'].value_counts()))
print('總體星等（清除空值）：{}'.format(df_del_na.groupby('餐廳名稱')['評論人給星'].sum()/df_del_na.groupby('餐廳名稱')['餐廳名稱'].value_counts()))
#兩年內、未清除
print('兩年內資料')
print('總體星等（未清除空值）：{}'.format(df_time_pro.groupby('餐廳名稱')['評論人給星'].sum()/df_time_pro.groupby('餐廳名稱')['餐廳名稱'].value_counts()))
print('總體星等（清除空值）：{}'.format(df_clean.groupby('餐廳名稱')['評論人給星'].sum()/df_clean.groupby('餐廳名稱')['餐廳名稱'].value_counts()))

所有資料：
總體星等（未清除空值）：餐廳名稱           餐廳名稱         
1010湘 (南港中信店)  1010湘 (南港中信店)    3.998632
112巷牛排         112巷牛排           3.833935
2018搞食堂        2018搞食堂          4.043956
285大排檔         285大排檔           3.735294
2J CAFE        2J CAFE          4.391667
                                  ...   
龍坊上海點心店        龍坊上海點心店          3.125000
龍洞海產           龍洞海產             3.961637
龍涎居雞膳食坊 大直店    龍涎居雞膳食坊 大直店      3.987395
龍翔園餐廳          龍翔園餐廳            4.093679
龟覓の鍋物          龟覓の鍋物            4.691781
Length: 973, dtype: float64
總體星等（清除空值）：餐廳名稱           餐廳名稱         
1010湘 (南港中信店)  1010湘 (南港中信店)    3.869249
112巷牛排         112巷牛排           3.755230
2018搞食堂        2018搞食堂          4.000000
285大排檔         285大排檔           3.586207
2J CAFE        2J CAFE          4.322222
                                  ...   
龍坊上海點心店        龍坊上海點心店          2.852632
龍洞海產           龍洞海產             3.972678
龍涎居雞膳食坊 大直店    龍涎居雞膳食坊 大直店      3.840000
龍翔園餐廳          龍翔園餐廳            4.183258
龟覓の鍋物          龟覓の鍋物    

In [195]:
#建立dataframe，依據“所有評論”、“有具體評論內容的評論”、“兩年內的所有評論”、“兩年內有具體評論內容的評論”分別計算各餐廳的星等
df_star = pd.DataFrame({'所有資料（未清除空值）': df.groupby('餐廳名稱')['評論人給星'].sum()/df.groupby('餐廳名稱')['餐廳名稱'].value_counts(),
                        '所有資料（清除空值）': df_del_na.groupby('餐廳名稱')['評論人給星'].sum()/df_del_na.groupby('餐廳名稱')['餐廳名稱'].value_counts(),
                        '兩年內資料（未清除空值)': df_time_pro.groupby('餐廳名稱')['評論人給星'].sum()/df_time_pro.groupby('餐廳名稱')['餐廳名稱'].value_counts(),
                        '兩年內資料（清除空值)': df_clean.groupby('餐廳名稱')['評論人給星'].sum()/df_clean.groupby('餐廳名稱')['餐廳名稱'].value_counts()
})

In [186]:
#單獨抓出每家餐廳的名稱
res_name = []
for i in df_star.index:
    res_name.append(i[0])

In [189]:
#把索引更換成餐廳名稱
df_star['餐廳名稱'] = res_name
df_star.set_index('餐廳名稱',inplace=True)

In [193]:
#匯出
df_star.to_csv('餐廳星等.csv')

抓出食物

In [141]:
#因為斷詞結果變成字串（"['名詞','名詞'...]"）因此要改回成列表的形式
def turn_to_list(x):
    text = re.sub('[\s\'\"\[\]]','',x)
    return text.split(',')

In [144]:
#把斷詞結果、詞性標註結果變成一維的列表
ws_result,pos_result = df_clean['斷詞結果'].map(turn_to_list), df_clean['詞性標注'].map(turn_to_list)

word_list = []
pos_list = []
for i in ws_result:
    word_list += i
for i in pos_result:
    pos_list += i

In [147]:
#利用pandas dataframe的filter來篩選出名詞
df_w2p = pd.DataFrame({'word':word_list,'pos':pos_list})

In [203]:
#計算不同名詞出現的頻率
w_n = df_w2p[df_w2p['pos']=='Na'].value_counts()

In [176]:
#把包含了斷詞結果、詞性、情感分析分數、情感分析分類的兩年內資料匯出
df_output = df_clean.drop(labels = ['Unnamed: 0','Unnamed: 0.1','Unnamed: 0.1.1'],axis = 1)
df_output.iloc[0:5].to_csv('分析結果格式.csv')
df_output.to_csv('分析結果.csv')

In [259]:
#如果傳入值在停用字當中就標記為1，沒有為0
#用來剔除找到的名詞中的停用字
def in_stop_word(x):
    stop_word = ['餐點','人', '味道', '環境', '口味', '菜', '態度', '人員', '份量', '價格', '食物', '食材',
                     '料理', '店員', '東西', '客人', '老闆', '整體', '店家', '氣氛', '口感', '菜色', '甜點',
                     '套餐', '感覺','朋友', '時間','價位', '飲料', '服務生', '品質', '地方', '菜單','餐', '空間',
                      '星', '水準', '速度','包廂','老闆娘','員工','車位','期間','阿姨','捷運','疫情'] 
    if x in stop_word:
        y =1
    else:
        y=0
    return y
#因為在處理時會以餐廳名稱做groupby
#使用groupby後的資料是dataframe的格式
#對傳入的dataframe先把名詞和詞性的部分拉成一維
def get_most_freq_word(x):
    word_list = []
    pos_list = []
    most_freq_word = []
    for word in x['斷詞結果']:
        word_list += word
    for pos in x['詞性標注']:
        pos_list += pos
    #利用datframe的filter找到名詞
    df_w2p = pd.DataFrame({'word':word_list,'pos':pos_list})
    #刪除停用字
    df_w2p['in_stop_word']= df_w2p['word'].map(in_stop_word)

    df_w2p = df_w2p.drop(df_w2p[df_w2p['in_stop_word']==1].index,axis=0)
    
    #計算不同名詞出現的頻率
    w_n = df_w2p[df_w2p['pos']=='Na'].value_counts()
    #抓出名詞
    for i,j,k in w_n.head(10).index:
        most_freq_word.append(i)
    #輸出常出現名詞列表
    return most_freq_word
    

In [260]:
tem_most_freq =df_output.groupby('餐廳名稱').apply(get_most_freq_word)
tem_most_freq.to_csv('每間餐廳評論中最常出現的詞.csv')